In [143]:
import os
import glob
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import cv2
from time import time

In [142]:
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %f" %(f.__name__,end-start))
        return r
    return wrapper

@_time
def stitchtiles(impths):
    # pre-allocate stitched image
    overlap = 0.1
    overlap = overlap/2 # cut this much from all sides of the image
    tilesz = Image.open(impths[0]).size
    pixel2adjust = round(tilesz[0]*overlap) # cut this much from all sides of the image
    cuttile = np.zeros(tilesz)[pixel2adjust:-pixel2adjust,pixel2adjust:-pixel2adjust]
    column = round(np.sqrt(tiles_per_well))
    row = round(np.sqrt(tiles_per_well))
    imstack = np.repeat(cuttile[np.newaxis,:, :], column, axis=0)
    imstack = np.repeat(imstack[np.newaxis, :, :, :], row, axis=0)
    # read tile into each grid
    for idx,im in enumerate(impths):
        rowidx = idx//4
        colidx = idx%4
        imstack[rowidx,colidx,:,:]=np.array(Image.open(im))[pixel2adjust:-pixel2adjust,pixel2adjust:-pixel2adjust]
    # re-order tiles for snake style
    imstack[1::2,:,:,:]=imstack[1::2,::-1,:,:]
    # stitch
    imstack = imstack.swapaxes(1,2)
    imstack = imstack.reshape(cuttile.shape[0]*row,cuttile.shape[1]*column)
    # normalize
    imstack_norm = cv2.normalize(imstack, None, alpha = 0, beta = 65535, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_16U)
    return imstack_norm

In [144]:
src=r'\\fatherserverdw\Bart\Stiffness\^Bleomycin Growth Curve\Plastic - Low Dose\Bio 3 - P6\20220309_GT22_Day0'
dst=os.path.join(src,'output')
ims = glob.glob(os.path.join(src,'*tif'))
tiles_per_well = 16
num_well = 24
for wellID in range(num_well):
    print(wellID+1,'/',num_well)
    ims2 = ims[wellID*tiles_per_well:wellID*tiles_per_well+tiles_per_well] # build for-loop in this line to process the entire folder
    imstack = stitchtiles(ims2)
    png = Image.fromarray(imstack)
    png.save(os.path.join(dst,'{}.png'.format(wellID)))

0 16
16 32
32 48
48 64
64 80
80 96
96 112
112 128
128 144
144 160
160 176
176 192
192 208
208 224
224 240
240 256
256 272
272 288
288 304
304 320
320 336
336 352
352 368
368 384
